<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install deep-daze stomp.py --upgrade

Requirement already up-to-date: deep-daze in /usr/local/lib/python3.7/dist-packages (0.10.2)
Requirement already up-to-date: stomp.py in /usr/local/lib/python3.7/dist-packages (7.0.0)


In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
 
flatten = itertools.chain.from_iterable

CMD = 'imagine --lower_bound_cutout 0.7 --deeper --image-width=384 --iterations=500 --epochs=5 --save-every=15 --save-progress=True --open_folder=False --overwrite=True "An intricate closeup drawing of an individual alien Neuron"'
WATCH_MEDIA_FILES = ["./*.jpg","./*.mp4"]

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)


!rm *.jpg

p = Popen(CMD, stdout = PIPE, 
        stderr = STDOUT, shell = True)

FOUND_MEDIA_FILES=[]
while True:
  line = p.stdout.readline()#
  if not line: break

  stdout.write(line)
  conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
  NEW_FOUND_MEDIA_FILES=list(flatten([glob(search_path) for search_path in WATCH_MEDIA_FILES]))
  NEW_FOUND_MEDIA_FILES.sort(key=os.path.getmtime)
  ADDED_MEDIA_FILES = list(set(NEW_FOUND_MEDIA_FILES)-set(FOUND_MEDIA_FILES))
  FOUND_MEDIA_FILES = NEW_FOUND_MEDIA_FILES

  if len(ADDED_MEDIA_FILES) > 0:
    for added in ADDED_MEDIA_FILES:
      sleep(0.1)
      if len(added) > 0:
        print("reading", added)
        with open(added,"rb") as media_file:
          added_media = media_file.read()
          base64_encoded_added = f"data:image/jpg;base64,{b64encode(added_media).decode('ascii')}"
        
        conn.send(body=base64_encoded_added, destination='/topic/colabMediaOut')
conn.disconnect()

epochs:   0%|          | 0/5 [00:00<?, ?it/s]
                                             
reading ./An_intricate_closeup_drawing_of_an_individual_alien_Neuron.jpg
loss: -27.55:   1%|          | 6/500 [00:22<31:00,  3.77s/it]
reading ./An_intricate_closeup_drawing_of_an_individual_alien_Neuron.000002.jpg
loss: -31.44:   3%|▎         | 14/500 [00:56<30:40,  3.79s/it]
                                             
reading ./An_intricate_closeup_drawing_of_an_individual_alien_Neuron.000001.jpg
loss: -40.79:   4%|▍         | 21/500 [01:19<30:22,  3.80s/it]
reading ./An_intricate_closeup_drawing_of_an_individual_alien_Neuron.000003.jpg
loss: -43.21:   6%|▌         | 29/500 [01:53<29:53,  3.81s/it]
                                             
loss: -46.33:   7%|▋         | 36/500 [02:16<29:26,  3.81s/it]
reading ./An_intricate_closeup_drawing_of_an_individual_alien_Neuron.000004.jpg
loss: -46.93:   8%|▊         | 38/500 [02:27<29:18,  3.81s/it]
